# Period analysis of cepheids from GAIA

In this notebook time-series analysis of variable stars from GAIA is performed.
First, GAIA database is queried for objects. Then, for each object, lightcurve is downloaded, and period is calculated.
Values for periods from GAIA database and those calculated here can be compared.

In [ ]:
import numpy as np
import os
import subprocess
import matplotlib.pyplot as plt
from astropy.io.votable import parse_single_table
from astroML.time_series import lomb_scargle, multiterm_periodogram, search_frequencies, lomb_scargle_BIC, lomb_scargle_bootstrap
from astroquery.gaia import Gaia

The first step is to load objects from external database. This is done in separate procedure.
In this case, the goal is to develop time-series analysis, so only few objects will be used.
Several cepheids are selected from GAIA catalog of variable stars. Also, only source ID and period are obtained.

In [ ]:
# get few sources where mode_best_classification = FUNDAMENTAL (i.e. no overtone or multimode pulsators)
query="select top 5 source_id, pf from gaiadr2.vari_cepheid where type_best_classification = 'ACEP' and mode_best_classification = 'FUNDAMENTAL'"

In [ ]:
name=None
output_file='results'
output_format='csv'
verbose=False
dump_to_file=True
background=False
upload_resource=None
upload_table_name=None

job = Gaia.launch_job_async(query, name, output_file, output_format, verbose, dump_to_file, background, upload_resource, upload_table_name)

results = job.get_results()

Optionally, we can view, save to file or plot the results.

In [ ]:
print("Results:\n", results)

# x = results['source_id']
# y = results['pf']
# scatterplot


Here, procedures that will be used in the main part are defined.

In [ ]:
def downloadLightcurveData(id, valid_data = 'True', band = 'G', format = 'VOTABLE', datadir = '/home/jaleksic/DATA/GaiaLCs/'):
    ''' Download lightcurve data from Gaia. Data is stored with the filename equal to source id.
    
    id (long):  Source_id value from the DR2 gaia_source catalogue.
    valid_data (boolean): Whether to return only valid data (data rows where flux is not null and rejected_by_photometry flag is not true) or all data associated to a given source
    band (string): Values = G | BP | RP
    format (string): Output format for the downloaded data. Values = VOTABLE | VOTABLE_PLAIN | FITS | CSV
    datadir: Output directory for download
    
    Note: at this moment only votable format is supported. Other formats will be added subsequently.
    '''
    if format=="VOTABLE": ext=".vot"
    elif format=="VOTABLE_PLAIN": ext=".votplain"
    elif format=="FITS": ext=".fits"
    elif format=="CSV": ext=".csv"
    else: print("Unsupported format")
    
    commandstring = "curl -k 'http://geadata.esac.esa.int/data-server/data?RETRIEVAL_TYPE=epoch_photometry&ID="+id+"&VALID_DATA="+valid_data+"&BAND="+band+"&FORMAT="+format+"' > "+datadir+id+ext
    subprocess.run(commandstring, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
def findBestFrequency(t, y, dy, LS_func=lomb_scargle, LS_kwargs=None, initial_guess=25, limit_fractions=[0.04,0.3,0.9,0.99], n_eval=10000, n_retry=5, n_save=50):
    omegas_iterative = search_frequencies(t, y, dy, LS_func=lomb_scargle, LS_kwargs=None, initial_guess=25, limit_fractions=[0.04,0.3,0.9,0.99], n_eval=10000, n_retry=5, n_save=50)
    bestFrequency = omegas_iterative[0][omegas_iterative[1].argmax()]
    return bestFrequency    

def analyzePeriodograms(file, plot_lightcurve=True, plot_LS=True, calculate_multiterm=False, plot_multiterm=False):
    ''' Take data from the file and analyze Lomb-Scargle and Multiterm periodograms
    file: file with lightcurve data
    '''
    
    table = parse_single_table(file) # Here, only votable format is supported. Other formats will be added.

    data = table.array
    
    time = data['time']
    flux = data['flux']
    flux_error = data['flux_error']
    
    if plot_lightcurve:
        # Plot the lightcurve
        ax1 = plt.subplot(311)
        ax1.scatter(time,flux, marker='.')
        ax1.set_title('Light curve')
        ax1.set_xlabel('time')
        ax1.set_ylabel('flux')
    
    
    tmax=max(time)
    tmin=min(time)
    tdata=tmax-tmin
    
    stepslice = 0.1
    omegamin = 2.*np.pi/tdata
    omegamax = 2.*np.pi*np.mean(1./np.ediff1d(time))
    omegastep = stepslice*omegamin 
    
    frequencies = np.arange(start=omegamin, stop=omegamax, step=omegastep)
    periods = 2*np.pi/frequencies
    
    # Calculate best frequency by iterative procedure
    bestfrequency = findBestFrequency(t=time, y=flux, dy=flux_error)
    bestperioditer = 2*np.pi/bestfrequency
    print("Best period calculated by iterative procedure:", bestperioditer)
    
    # Calculate Lomb-Scargle periodogram
    P_LS = lomb_scargle(t=time, y=flux, dy=flux_error, omega=frequencies, generalized=True, subtract_mean=True, significance=None)
    
    # Find the best period from LS periodogram
    bestperiodpls = periods[P_LS.argmax()]
    print("Best period read from LS periodogram:", bestperiodpls)

    # Get significance via bootstrap
    D = lomb_scargle_bootstrap(t=time, y=flux, dy=flux_error, omega=frequencies, generalized=True, N_bootstraps=100, random_state=0)
    sig1, sig5 = np.percentile(D, [99, 95])

    if plot_LS:
        # Plot Lomb-Scargle periodogram & significance levels
        ax2 = plt.subplot(312)
        ax2.plot(periods,P_LS,'.-')
        ax2.plot([periods[0], periods[-1]], [sig1, sig1], ':', c='black')
        ax2.plot([periods[0], periods[-1]], [sig5, sig5], ':', c='grey')
        ax2.set_title("Lomb-Scargle periodogram")
        ax2.set_xlabel("periods")
        ax2.set_ylabel("P ls")
    

    if calculate_multiterm:
        # Calculate Multiterm periodogram
        P_M = multiterm_periodogram(t=time, y=flux, dy=flux_error, omega=frequencies, n_terms=3)
    
    if plot_multiterm:
        # Plot Multiterm periodogram
        ax3 = plt.subplot(313)
        ax3.plot(frequencies,P_M,'.-')
        ax3.set_title("Multiterm periodogram")
        ax3.set_xlabel("frequencies")
        ax3.set_ylabel("P m")

    if plot_lightcurve or plot_LS or plot_multiterm:
        # Plots
        plt.suptitle(file)
        plt.subplots_adjust(hspace=0.4)
        plt.show()
    

Main part. For each object (source_id), lightcurve data are downloaded (time, flux, flux_error). Then, analysis is performed.

In [ ]:
path = '/home/jaleksic/DATA/GaiaLCs/'

for i in results:
    print("=========================")
    print("Processing",i['source_id'])
    downloadLightcurveData(str(i['source_id']))
    print("Observed period",i['pf'])
    analyzePeriodograms(path+str(i['source_id'])+'.vot')

Values for periods at GAIA database are calculated using the Levenberg-Marquardt non linear ﬁtting algorithm. Values in this notebook are calculated using astroML package, which uses astropy's Lomb-Scargle algorithm. Values obtained from different procedures can be compared.
(Clementini et al. 2016, A&A, 595, A133)
If plot_LS is set to True, one can examine the position of the highest peak (zoom is needed).
Also, in this example, only sources with mode_best_classification = FUNDAMENTAL are analyzed. Next step is to consider  overtones and multimode pulsators.